In [1]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
with open("text_data.pickle", "rb") as file:
    text_data = pickle.load(file)

In [3]:
all_text = ''.join(text_data)

In [4]:
id2char = sorted(set(all_text))
len(id2char)

478

In [5]:
char2id = {c:i for i, c in enumerate(id2char)}

In [6]:
all_text_as_int = np.array([char2id[c] for c in all_text])

In [7]:
seq_length = 400
examples_per_epoch = len(all_text)

char_dataset = tf.data.Dataset.from_tensor_slices(all_text_as_int)

for i in char_dataset.take(10):
    print(id2char[i.numpy()])

Г
р
и
г
о
р
и
а
н
с


In [8]:
id2char = np.array(id2char)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(id2char[item.numpy()])))

'Григорианският календар (понякога наричан и Грегориански календар, „нов стил“) е съвременният международно признат светски календар, на който се основава и международният стандарт ISO 8601.\n\nГригорианският календар е въведен в употреба на 4 октомври 1582\xa0г. в съответствие с була от 24 февруари 1582\xa0г. на папа Григорий XIII, чието име носи и днес. Той поправя древноримския Юлиански календар, като в '
'него са нанесени някои корекции, за да се отчете по-точно дължината на тропическата година. В Юлианския календар се приема, че времето между две последователни пролетни равноденствия е 365,25 дни и дробната част се компенсира, като една на всеки четири години има с един ден повече (високосна година). При Григорианския календар годините, кратни на 100 не са високосни (с изключение на годините, кратн'
'и на 400, т.е. на всеки четири века се пропускат три високосни години, като например 1700, 1800 и 1900 година), което дава средна продължителност на годината от 365,2425 дни. Тази сто

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(id2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(id2char[target_example.numpy()])))

Input data:  'Григорианският календар (понякога наричан и Грегориански календар, „нов стил“) е съвременният международно признат светски календар, на който се основава и международният стандарт ISO 8601.\n\nГригорианският календар е въведен в употреба на 4 октомври 1582\xa0г. в съответствие с була от 24 февруари 1582\xa0г. на папа Григорий XIII, чието име носи и днес. Той поправя древноримския Юлиански календар, като в'
Target data: 'ригорианският календар (понякога наричан и Грегориански календар, „нов стил“) е съвременният международно признат светски календар, на който се основава и международният стандарт ISO 8601.\n\nГригорианският календар е въведен в употреба на 4 октомври 1582\xa0г. в съответствие с була от 24 февруари 1582\xa0г. на папа Григорий XIII, чието име носи и днес. Той поправя древноримския Юлиански календар, като в '


In [11]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 400), (64, 400)), types: (tf.int32, tf.int32)>

In [25]:
embedding_dim = 256
rnn_units = 1024


def build_model(batch_size=64):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(len(id2char), 
                                        embedding_dim, 
                                        batch_input_shape=[batch_size, None]))
    model.add(tf.keras.layers.GRU(rnn_units, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dense(len(id2char)))
    return model

model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           122368    
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 478)           489950    
Total params: 4,550,622
Trainable params: 4,550,622
Non-trainable params: 0
_________________________________________________________________


In [26]:
for in_batch, tar_batch in dataset.take(1):
    pred_batch = model(in_batch)
    print(pred_batch.shape)

(64, 400, 478)


In [27]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [45]:
import os

checkpoint_dir = "./char_training"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = os.path.join(checkpoint_dir, "ckpt_{epoch}"),
    save_weights_only=True
)

In [46]:
def generate_text(model, start_string, num_generate = 1000):

    input_eval = [char2id[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(id2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [47]:
class PrintGenText(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if (epoch>0):
            curr_model = build_model(1)
            curr_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
            curr_model.build(tf.TensorShape([1, None]))
            print(generate_text(curr_model, "В началото на", 200))
        
gen_callback = PrintGenText()

In [48]:
history = model.fit(dataset, shuffle=True, epochs=5, callbacks = [checkpoint_callback, gen_callback])

Train for 31026 steps
Epoch 1/5
31026/31026 [==============================] - 10102s 326ms/step - loss: 0.7850
В началото на 2010 г. и първото турнирно наследство са женени.

В Египет
 
Фрегатюра Катулари се адриатира. Присъединява
I'i всички кандидатори основават медиите от ракети в потушаванието на Асирия (Руско-турската
Epoch 2/5
31026/31026 [==============================] - 14862s 479ms/step - loss: 1.2451
В началото най-голите – „Нюжаг.Дара
Капата оковскича по-двръжко корподмистичести на Хайския, портигтао, Салифануа МилошхавниВ ()
ТМергарТФрижда, Кеймжнова.

През 1821 годи
Буйса фон Йонферсеменската Муитфлания
Геж
Epoch 3/5
12214/31026 [==========>...................] - ETA: 1:41:27 - loss: 1.3041

KeyboardInterrupt: 